Run third

### Generowanie predykcji dla użytkowników

Import niezbędnych bibliotek, definicja ustawień logowania i stałych

In [56]:
import pandas as pd

import logging as log
from os.path import dirname, join, abspath
import sys
sys.path.insert(0, abspath(join(dirname('recommender_system'), '..')))

from src.utils.helper import (
    create_table_in_postgres_db,
    load_recommendations_data_from_csv_to_postgres_table,
    generate_recommendations,
    get_newest_existing_model_version,
    save_data_to_pkl,
    save_recommendations_to_csv,
    unpickle
)

In [57]:
logger = log.getLogger()
logger.setLevel(log.INFO)

In [58]:
BASE_PATH = abspath(join(dirname('recommender_system'), '../../..'))
RESULTS_PATH = f'{BASE_PATH}/results'

Załadowanie danych z plików do obiektów DataFrame

In [59]:
ratings_df = pd.read_csv(f'{BASE_PATH}/input_data/All_Beauty.csv')
items_df = pd.read_csv(f'{BASE_PATH}/input_data/meta_All_Beauty.csv')

Stworzenie listy użytkowników i produktów

In [61]:
# Drop duplicated user ids
log.info(f"Duplicated users: {ratings_df.duplicated(subset='reviewerID').sum()}.")
ratings_df = ratings_df.drop_duplicates(subset='reviewerID')
log.info(f"Rows number after data cleaning: {len(ratings_df)}.")

INFO:root:Duplicated users: 47307.
INFO:root:Rows number after data cleaning: 324038.


In [62]:
user_ids = list(ratings_df['reviewerID'])
asins = list(items_df["asin"])
print(f"Number of user ids: {len(user_ids)}\nNumber of item ids: {len(asins)}")

Number of user ids: 324038
Number of item ids: 32488


Stworzenie tabeli w bazie danych

In [7]:
## Create table to store ratings data if it does not exist
create_recommendations = '''create table IF NOT exists recommendations (
	user_id varchar(20),
    recommendations varchar,
    model varchar(100),
    model_version int4
);'''

create_table_in_postgres_db(create_recommendations)

INFO:root:Query executed


#### Generowanie rekomendacji z modelu z mechanizmem optymalizacji adagrad oraz funkcją straty BPR

In [9]:
model_name = "adagrad_bpr_model"

model_results_path = f'{RESULTS_PATH}/{model_name}'
version = get_newest_existing_model_version(model_results_path)

dataset = unpickle(f'{RESULTS_PATH}/{model_name}/dataset_v{version}.pkl')
model = unpickle(f'{RESULTS_PATH}/{model_name}/{model_name}_v{version}.pkl')

recommendations = generate_recommendations(dataset, asins, model, user_ids)

In [7]:
RECOMMENDATIONS_FILE_PATH = f'{RESULTS_PATH}/recommendations/recommendations_{model_name}_v{version}'

Zapisywanie do pliku pickle

In [11]:
save_data_to_pkl(f'{RECOMMENDATIONS_FILE_PATH}.pkl', recommendations)

INFO:root:File /home/mborettini/recommender_system/results/recommendations/recommendations_adagrad_bpr_model_v1.pkl saved


Zapisywanie do pliku csv

In [12]:
save_recommendations_to_csv(
    model_name,
    f'{RECOMMENDATIONS_FILE_PATH}.csv',
    recommendations,
    version
)

INFO:root:File saved


Ładowanie pliku csv do bazy danych PostgresSQL

In [9]:
model_name = "adagrad_bpr_model"
version = 1
load_recommendations_data_from_csv_to_postgres_table(model_name, f'{RECOMMENDATIONS_FILE_PATH}.csv', version)

INFO:root:Loaded 324038 rows to recommendations table


#### Generowanie rekomendacji z modelu z mechanizmem optymalizacji adadelta oraz funkcją straty BPR

In [10]:
model_name = "adadelta_bpr_model"

model_results_path = f'{RESULTS_PATH}/{model_name}'
version = get_newest_existing_model_version(model_results_path)

dataset = unpickle(f'{RESULTS_PATH}/{model_name}/dataset_v{version}.pkl')
model = unpickle(f'{RESULTS_PATH}/{model_name}/{model_name}_v{version}.pkl')

recommendations = generate_recommendations(dataset, asins, model, user_ids)

Zapisywanie do pliku pickle

In [11]:
save_data_to_pkl(f'{RECOMMENDATIONS_FILE_PATH}.pkl', recommendations)

INFO:root:File /home/mborettini/recommender_system/results/recommendations/recommendations_adagrad_bpr_model_v1.pkl saved


Zapisywanie do pliku csv

In [12]:
save_recommendations_to_csv(
    model_name,
    f'{RECOMMENDATIONS_FILE_PATH}.csv',
    recommendations,
    version
)

INFO:root:File saved


Ładowanie pliku csv do bazy danych PostgresSQL

In [13]:
load_recommendations_data_from_csv_to_postgres_table(model_name, f'{RECOMMENDATIONS_FILE_PATH}.csv', version)

INFO:root:Loaded 324038 rows to recommendations table


#### Generowanie rekomendacji z modelu z mechanizmem optymalizacji adagrad oraz funkcją straty WARP

In [ ]:
model_name = "adagrad_warp_model"

model_results_path = f'{RESULTS_PATH}/{model_name}'
version = get_newest_existing_model_version(model_results_path)

dataset = unpickle(f'{RESULTS_PATH}/{model_name}/dataset_v{version}.pkl')
model = unpickle(f'{RESULTS_PATH}/{model_name}/{model_name}_v{version}.pkl')

recommendations = generate_recommendations(dataset, asins, model)

Zapisywanie do pliku pickle

In [ ]:
save_data_to_pkl(f'{RECOMMENDATIONS_FILE_PATH}.pkl', recommendations)

Zapisywanie do pliku csv

In [ ]:
save_recommendations_to_csv(
    model_name,
    f'{RECOMMENDATIONS_FILE_PATH}.csv',
    recommendations,
    version
)

Ładowanie pliku csv do bazy danych PostgresSQL

In [14]:
load_recommendations_data_from_csv_to_postgres_table(model_name, f'{RECOMMENDATIONS_FILE_PATH}.csv', version)

INFO:root:Loaded 324038 rows to recommendations table


#### Generowanie rekomendacji z modelu z mechanizmem optymalizacji adadelta oraz funkcją straty WARP

In [9]:
model_name = "adadelta_warp_model"

model_results_path = f'{RESULTS_PATH}/{model_name}'
version = get_newest_existing_model_version(model_results_path)

dataset = unpickle(f'{RESULTS_PATH}/{model_name}/dataset_v{version}.pkl')
model = unpickle(f'{RESULTS_PATH}/{model_name}/{model_name}_v{version}.pkl')

recommendations = generate_recommendations(dataset, asins, model, user_ids)

INFO:root:Data have been pickled.


Zapisywanie do pliku pickle

In [ ]:
save_data_to_pkl(f'{RECOMMENDATIONS_FILE_PATH}.pkl', recommendations)

Zapisywanie do pliku csv

In [ ]:
save_recommendations_to_csv(
    model_name,
    f'{RECOMMENDATIONS_FILE_PATH}.csv',
    recommendations,
    version
)

Ładowanie pliku csv do bazy danych PostgresSQL

In [ ]:
load_recommendations_data_from_csv_to_postgres_table(model_name, f'{RECOMMENDATIONS_FILE_PATH}.csv', version)

#### Generowanie rekomendacji z modelu z mechanizmem optymalizacji adagrad oraz funkcją straty k-OS WARP

In [10]:
model_name = "adagrad_kos_warp_model"

model_results_path = f'{RESULTS_PATH}/{model_name}'
version = get_newest_existing_model_version(model_results_path)

dataset = unpickle(f'{RESULTS_PATH}/{model_name}/dataset_v{version}.pkl')
model = unpickle(f'{RESULTS_PATH}/{model_name}/{model_name}_v{version}.pkl')

recommendations = generate_recommendations(dataset, asins, model, user_ids)

INFO:root:Data have been pickled.


Zapisywanie do pliku pickle

In [ ]:
save_data_to_pkl(f'{RECOMMENDATIONS_FILE_PATH}.pkl', recommendations)

Zapisywanie do pliku csv

In [ ]:
save_recommendations_to_csv(
    model_name,
    f'{RECOMMENDATIONS_FILE_PATH}.csv',
    recommendations,
    version
)

Ładowanie pliku csv do bazy danych PostgresSQL

In [ ]:
load_recommendations_data_from_csv_to_postgres_table(model_name, f'{RECOMMENDATIONS_FILE_PATH}.csv', version)

Generowanie rekomendacji z modelu z mechanizmem optymalizacji adadelta oraz funkcją straty k-OS WARP

In [11]:
model_name = "adadelta_kos_warp_model"

model_results_path = f'{RESULTS_PATH}/{model_name}'
version = get_newest_existing_model_version(model_results_path)

dataset = unpickle(f'{RESULTS_PATH}/{model_name}/dataset_v{version}.pkl')
model = unpickle(f'{RESULTS_PATH}/{model_name}/{model_name}_v{version}.pkl')

recommendations = generate_recommendations(dataset, asins, model, user_ids)

INFO:root:Data have been pickled.


Zapisywanie do pliku pickle

In [ ]:
save_data_to_pkl(f'{RECOMMENDATIONS_FILE_PATH}.pkl', recommendations)

Zapisywanie do pliku csv

In [ ]:
save_recommendations_to_csv(
    model_name,
    f'{RECOMMENDATIONS_FILE_PATH}.csv',
    recommendations,
    version
)

Ładowanie pliku csv do bazy danych PostgresSQL

In [ ]:
load_recommendations_data_from_csv_to_postgres_table(model_name, f'{RECOMMENDATIONS_FILE_PATH}.csv', version)